<a href="https://colab.research.google.com/github/aknip/Autogen-AutoGPT/blob/main/AutoGen_Screen_Writer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autogen Scene Writer
How to:

- lorem
- ipsum

Source: https://github.com/abhilashi/ai-explorations/blob/main/ai_scene_writer.py

In [1]:
import json
import os
from getpass import getpass
import psutil
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

Secrets (JSON string): ··········


In [ ]:
!pip install pyautogen~=0.1.0 docker

## Setup




In [17]:
from autogen import AssistantAgent, UserProxyAgent
import autogen

config_list = [{'model': 'gpt-4', 'api_key': CREDS['OpenAI']['v1']['credential']}]

llm_config = {
    "seed": 43,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "request_timeout": 120,
}

#print(config_list)

screen_writer = autogen.UserProxyAgent(
    name="screen_writer",
    system_message="A human screen writer. Dialoge needs to be approved by this screen writer.",
    code_execution_config={"work_dir": "work"},
    #code_execution_config=False,
)

# Alternative to save result to file - not working...!
#screen_writer = autogen.UserProxyAgent(
#    name="screen_writer",
#    human_input_mode="TERMINATE",
#    max_consecutive_auto_reply=10,
#    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
#    code_execution_config={"work_dir": "web"},
#    llm_config=llm_config,
#    system_message="A human screen writer. Dialoge needs to be approved by this screen writer.",
#)


monica_mingle = autogen.AssistantAgent(
    name="monica_mingle",
    llm_config=llm_config,
    system_message='''Monica Mingle. Inspired by Monica Geller, she's a perfectionist and an obsessive-compulsive chef. She's always trying to host the perfect party and is competitive to a fault.
''',
)

joey_jumble = autogen.AssistantAgent(
    name="joey_jumble",
    llm_config=llm_config,
    system_message='''Based on Joey Tribbiani, he's a lovable, not-so-bright actor who often gets his words mixed up. He's known for his catchphrase, "How you doin'?", and has an insatiable appetite, especially for pizza
''',
)
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check script, make sure the dialogue between monica mingle and joey jumble is hillarious. provide feedback.",
    llm_config=llm_config,
)

groupchat = autogen.GroupChat(agents=[
                              screen_writer, monica_mingle, joey_jumble, critic], messages=[], max_round=5)
manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config,
    #code_execution_config={"work_dir": "work"}
    )



# Start Chat

In [19]:
screen_writer.initiate_chat(
    manager,
    message="""
screen write a birthday party. Save the final text as file party.txt
""",
)

screen_writer (to chat_manager):


screen write a birthday party. Save the final text as file party.txt


--------------------------------------------------------------------------------
monica_mingle (to chat_manager):

INT. MONICA'S APARTMENT - DAY

Monica Mingle, a perfectionist and obsessive-compulsive chef, is bustling around her apartment. The place is spotless, every surface gleaming. She's preparing for a birthday party.

MONICA
(looking at her checklist)
Okay, appetizers, main course, dessert, drinks... Everything's perfect.

She moves to the living room, adjusting the decorations. She straightens a banner that reads "HAPPY BIRTHDAY!" and fluffs up the balloons.

Suddenly, the door swings open. It's JOEY TRIBBIANI, a lovable, not-so-bright actor who often gets his words mixed up.

JOEY
(smiling)
Monica, are you trying to make me fall in love with you? Because this is how you do it!

Monica laughs, rolling her eyes at Joey's twist on his catchphrase.

MONICA
I'm doing great, Jo